# Mounting Drive


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Showing sample images from the Dataset

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Set paths to the dataset folders
dataset_path = "/content/drive/MyDrive/Constellation Datasest"
stars_folder = os.path.join(dataset_path, "stars")
labels_folder = os.path.join(dataset_path, "labels")

# Get the list of image files in both folders
stars_files = sorted([f for f in os.listdir(stars_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])
labels_files = sorted([f for f in os.listdir(labels_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])

if len(stars_files) != len(labels_files):
    print("Warning: Number of images in 'stars' and 'labels' folders do not match!")

num_stars_to_show = int(input("Enter the number of star images to display: "))
num_stars_to_show = min(num_stars_to_show, len(stars_files), len(labels_files))

for i, (star_file, label_file) in enumerate(zip(stars_files, labels_files)):
    if i >= num_stars_to_show:
        break

    star_path = os.path.join(stars_folder, star_file)
    label_path = os.path.join(labels_folder, label_file)

    star_image = cv2.imread(star_path)
    label_image = cv2.imread(label_path)

    star_image = cv2.cvtColor(star_image, cv2.COLOR_BGR2RGB)
    label_image = cv2.cvtColor(label_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title("Star")
    plt.imshow(star_image)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Label")
    plt.imshow(label_image)
    plt.axis("off")

    plt.show()


# Connecting Star points Using Nearest Neighbours

In [ ]:
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors
from google.colab.patches import cv2_imshow

# Upload an image

image_path = "/content/drive/MyDrive/Constellation Datasest/stars/76.png"  # Get the uploaded file name

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

_, thresh = cv2.threshold(image, 230, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Extract contour centers
contour_centers = []
for contour in contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        contour_centers.append((cx, cy))

contour_centers = np.array(contour_centers)

knn = NearestNeighbors(n_neighbors=3)  # 2 includes the point itself
knn.fit(contour_centers)
connections = knn.kneighbors(contour_centers, return_distance=False)

height, width = image.shape
black_image = np.ones((height, width, 3), dtype=np.uint8) * 255

# Draw connections
for i, neighbors in enumerate(connections):
    point1 = contour_centers[i]
    for neighbor_index in neighbors[1:]:  # Skip the first neighbor (itself)
        point2 = contour_centers[neighbor_index]
        cv2.line(black_image, tuple(point1), tuple(point2), (0, 0, 0), 10)

cv2_imshow(image)
cv2_imshow(black_image)

cv2.imwrite("connections_result.jpg", black_image)

# Streamlit Graphical Interface

In [ ]:
import streamlit as st
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors
from PIL import Image


def process_image(image_path, threshold_value, n_neighbors):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    _, thresh = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contour_centers = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            contour_centers.append((cx, cy))

    contour_centers = np.array(contour_centers)

    knn = NearestNeighbors(n_neighbors=n_neighbors)
    knn.fit(contour_centers)
    connections = knn.kneighbors(contour_centers, return_distance=False)

    height, width = image.shape
    black_image = np.ones((height, width, 3), dtype=np.uint8) * 255

    for i, neighbors in enumerate(connections):
        point1 = contour_centers[i]
        for neighbor_index in neighbors[1:]:  # Skip the first neighbor (itself)
            point2 = contour_centers[neighbor_index]
            cv2.line(black_image, tuple(point1), tuple(point2), (0, 0, 0), 10)

    return image, black_image


# Streamlit UI
def main():
    st.title("Constellation Finder")

    # File uploader
    uploaded_file = st.file_uploader("Upload an Image", type=["png", "jpg", "jpeg"])

    if uploaded_file is not None:
        # Load the uploaded image
        image_path = uploaded_file.name
        with open(image_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Threshold slider
        threshold_value = st.slider("Threshold Value", min_value=0, max_value=255, value=230, step=1)

        # Number of neighbors slider
        n_neighbors = st.slider("Number of Neighbors", min_value=2, max_value=10, value=3, step=1)

        original_image, processed_image = process_image(image_path, threshold_value, n_neighbors)

        col1, col2 = st.columns(2)
        with col1:
            st.image(original_image, caption="Original Image", use_column_width=True, channels="GRAY")
        with col2:
            st.image(processed_image, caption="Processed Image", use_column_width=True)


if __name__ == "__main__":
    main()


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py

# Abstract Art Image Generation

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image
import torch
import numpy as np
import cv2

input_image_path = "/content/connections_result.jpg"
input_image = Image.open(input_image_path).convert("RGB")
input_image = input_image.resize((512, 512))  # Resize for the model

skeleton_np = np.array(input_image)
edges = cv2.Canny(skeleton_np, 100, 200)  # Edge detection
edges_img = Image.fromarray(edges)

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", controlnet=controlnet,
                                                         torch_dtype=torch.float16)
pipe.to("cuda")

prompt = "abstract art"
output = pipe(prompt, image=edges_img, num_inference_steps=50).images[0]

output.save("output.png")
output.show()